# Random Forest

The [Proteomics Data Mining Challenge 2020](https://proteomicsnews.blogspot.com/2020/02/proteomics-data-mining-challenge-2020.html) was a community challenge to (re)analyze public amyotrophic lateral sclerosis (ALS) proteomics to find putative biomarkers. ALS is a progressive nervous system disease that affects nerve cells in the brain and spinal cord, causing loss of muscle control, eventually leading to death.

Proteomics data for 33 individuals with ALS and 30 healthy controls were analyzed by open modification searching using the ANN-SoLo spectral library search engine. Protein abundances were calculated using spectral counting.

**Can we build a random forest classifier to predict ALS disease state? Can we derive relevant biomarkers from the random forest model?**

Start by loading the features file and explore the protein abundance data.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, VarianceThreshold
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
# Read the data table:
X = pd.read_csv("../data/als_features.csv", index_col="filename")
# Read the class labels:
y = pd.read_csv("../data/als_filenames.csv", index_col="filename")
# Make sure both files are sorted in the same order:
y = y.reindex(index=X.index)

In [ ]:
print(f"Number of samples: {len(X)}")
print(f"Number of features: {len(X.columns):,}")

In [ ]:
X.head()

We have a lot more features $k$ than samples $n$. This is very common when analyzing molecular data, where thousands to tens of thousands of analytes can be measured in a single experiment.

**What might happen for data with $k >> n$? Which kind of machine learning solutions do we have and what are their strengths/weaknesses?**

Now we will train a random forest classifier. To evaluate the model performance we will split the data into a train, validation, and test split.

In [ ]:
# Split into train and test set:
X_train, X_test, y_train, y_test = train_test_split(
    X.values,
    y["label"].map({"A": 0, "H": 1}).values,
    test_size=11,
    random_state=0,
)
# Further split the train set into a train and validation set:
X_train, X_val, y_train, y_val = train_test_split(
    X_train,
    y_train,
    test_size=11,
    random_state=0,
)

print(f"Number of samples in the train set: {X_train.shape[0]}")
print(f"Number of samples in the validation set: {X_val.shape[0]}")
print(f"Number of samples in the test set: {X_test.shape[0]}")

We will build a random forest model including three different types of feature selection:

- No feature selection.
- Select the $k'$ best features using a statistical test.
- Reduce the dimensionality to $k'$ using PCA.

In [ ]:
# Create a model that combines preprocessing and an RF classifier:
k = 20
model = make_pipeline(
    StandardScaler(),
    VarianceThreshold(),
    # Uncomment the following lines one by one:
    # No feature selection: Don't uncomment anything.
    #     SelectKBest(k=k),
    #     PCA(n_components=k, random_state=0),
    RandomForestClassifier(random_state=0),
)

# Train the model on the train set:
model.fit(X_train, y_train)
# Evaluate the model on the validation set:
y_pred = model.predict_proba(X_val)
print(f"Validation AUC = {roc_auc_score(y_val, y_pred[:, 1]):.3f}")

Write down the validation AUC for each of the different models:

- No feature selection: AUC = ???
- $k$ best feature selection: AUC = ???
- PCA: AUC = ???

**Which method achieved the best performance? Why?**

Bonus: Try different values for $k$ for PCA and best feature selection. Which AUC scores do you get? What does this indicate?

Now that we have completed model selection, evaluate the best model on the test set. Remember, we only do this at the very end of our modeling!

In [ ]:
# Copy the final model specification here:
model = make_pipeline(
    # ...
    StandardScaler(),
    VarianceThreshold(),
)

# Retrain the final model on the train set:
model.fit(X_train, y_train)
# Evaluate the final model on the test set:
y_pred = model.predict_proba(X_test)
auc_test = roc_auc_score(y_test, y_pred[:, 1])
fpr, tpr, _ = roc_curve(y_test, y_pred[:, 1])
print(f"Test AUC = {auc_test:.3f}")

In [ ]:
# Let's look at the ROC curve:
fig, ax = plt.subplots()

ax.plot(fpr, tpr, label=f"AUC (test) = {auc_test:.3f}", marker="o")
ax.plot([0, 1], [0, 1], c="black", ls="--")
ax.legend(loc="lower right", frameon=False)

ax.set_xlim(0, 1.05)
ax.set_ylim(0, 1.05)

ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")

sns.despine(ax=ax)

plt.show()
plt.close()

What is the model performance on the test set compared to model performance on the validation set? Can you explain this?

**Were we able to successfully train a classifier to predict ALS status?**

**What are strategies to further improve model performance?**

## Feature importance

Information on the most important features used in the random forest can provide insights into relevant biomarkers. Random forest feature importances are computed as the mean and standard deviation of accumulation of the impurity decrease within each tree.

**Let's look at the most important features.**

In [ ]:
# Extract the retained features after feature selection:
selected_features = X.columns[model["variancethreshold"].get_support()]
selected_features = selected_features[model["selectkbest"].get_support()]
# Match features to feature importances:
feature_importances = pd.Series(
    model["randomforestclassifier"].feature_importances_,
    index=selected_features,
)
# Sort by descending feature importance:
feature_importances = feature_importances.sort_values(ascending=False)

fig, ax = plt.subplots()

feature_importances.plot.bar(legend=False, ax=ax)

ax.set_ylim(0, ax.get_ylim()[1])

ax.set_xlabel("Protein")
ax.set_ylabel("Feature importance")

sns.despine()

plt.show()
plt.close()